In [1]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

# fetch dataset
covertype = fetch_ucirepo(id=31)

# data (as pandas dataframes)
X = covertype.data.features
y = covertype.data.targets

# metadata
print(covertype.metadata)

# variable information
print(covertype.variables)


{'uci_id': 31, 'name': 'Covertype', 'repository_url': 'https://archive.ics.uci.edu/dataset/31/covertype', 'data_url': 'https://archive.ics.uci.edu/static/public/31/data.csv', 'abstract': 'Classification of pixels into 7 forest cover types based on attributes such as elevation, aspect, slope, hillshade, soil-type, and more.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 581012, 'num_features': 54, 'feature_types': ['Categorical', 'Integer'], 'demographics': [], 'target_col': ['Cover_Type'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1998, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C50K5N', 'creators': ['Jock Blackard'], 'intro_paper': None, 'additional_info': {'summary': 'Predicting forest cover type from cartographic variables only (no remotely sensed data).  The actual forest cover type for a given observation (30 x 30 meter cell) was determined from

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# no missing values and things are already encoded

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Identify non-binary columns
non_binary_cols = []
for col in X_train.columns:
    if len(X_train[col].unique()) > 2:
        non_binary_cols.append(col)

# Feature scaling (standardization) for non-binary columns
scaler = StandardScaler()
X_train[non_binary_cols] = scaler.fit_transform(X_train[non_binary_cols])
X_test[non_binary_cols] = scaler.transform(X_test[non_binary_cols])

X_train

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4
519924,1.177860,-1.194008,0.654794,-0.137989,0.800190,-0.411853,-0.267398,-1.381834,-0.536634,0.465719,...,0,0,0,0,0,0,0,0,1,0
318451,0.014050,-1.202944,0.521223,-0.636898,-0.332415,-0.710007,-0.230026,-1.179491,-0.405885,-0.634310,...,0,0,0,0,0,0,0,0,1,0
22325,0.278228,0.262598,-0.680920,-0.669845,-0.624146,2.636373,0.367937,1.147457,0.404758,0.796934,...,0,0,0,0,0,0,0,0,0,0
449376,0.549546,0.423450,-0.146634,-0.373324,-0.418218,-0.144476,0.031583,1.400386,0.744704,-0.943646,...,0,0,0,0,0,0,0,0,1,0
482753,0.603095,1.701332,-0.680920,-0.702792,-0.177968,-1.153071,-0.342144,0.135741,0.483207,-0.797277,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,0.795873,-0.765069,-0.146634,0.436226,-0.109326,0.411437,0.816409,-0.471289,-0.876580,1.777001,...,0,0,0,0,0,0,0,0,0,0
259178,0.760174,0.003447,1.990508,2.102394,4.198005,-0.767073,0.928527,0.236912,-1.164228,0.646039,...,0,0,0,0,0,0,0,0,0,0
365838,0.692344,1.174094,0.387652,0.309146,-0.092165,-0.674742,-1.538069,0.692185,1.633796,0.048492,...,0,0,0,0,0,0,0,0,1,0
131932,0.378187,1.719204,0.921937,-0.683965,-0.469700,1.522623,-1.313833,-1.078319,0.430907,-1.023621,...,0,0,0,0,0,0,0,0,0,0


In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

(464809, 54)
(116203, 54)
(464809, 1)
(116203, 1)


In [5]:
nb_classes = 7
y_train=y_train-1
y_test=y_test-1
Y_train = to_categorical(y_train, nb_classes)
Y_test = to_categorical(y_test, nb_classes)

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


# # Define the MLP model
# model = Sequential()
# model.add(Dense(32, activation='relu', input_shape=(54,)))  # Input layer
# model.add(Dense(32, activation='relu'))  # Hidden layer
# model.add(Dense(7, activation='softmax'))  # Output layer (7 classes)

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.001), metrics=['accuracy'])


# # Train the model
# model.fit(X_train, Y_train, epochs=10, batch_size=128, validation_split=0.1)  # Adjust epochs and batch_size as needed

# # Evaluate the model
# loss, accuracy = model.evaluate(X_test, Y_test)
# print(f"Test Loss: {loss:.4f}")
# print(f"Test Accuracy: {accuracy:.4f}")


In [9]:
from tensorflow.keras.layers import Dense, Input, Add, Activation
from tensorflow.keras.models import Sequential, Model

# Input layer
input_tensor = Input(shape=(X_train.shape[1],))

# Initial layers
x = Dense(128, activation='relu')(input_tensor)
x = Dense(64, activation='relu')(x)

# Custom Residual Block
skip = x  # Store input for skip connection
residual = Dense(64, activation='relu')(x)
residual = Dense(64, activation='relu')(residual)

# Linear projection for residual connection (if dimensions differ)
if residual.shape[-1] != skip.shape[-1]:
    skip = Dense(residual.shape[-1])(skip)

residual = Add()([residual, skip])
residual = Activation('relu')(residual)

# Additional Skip Connection
x = Add()([x, residual]) #adding skip connection output to previous layer's output
x = Activation('relu')(x)

# Final Layers
x = Dense(32, activation='relu')(x)
output_tensor = Dense(7, activation='softmax')(x)  # Output layer (7 classes)

# Create the model
model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=128, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/10
3269/3269 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.7474 - loss: 0.6055 - val_accuracy: 0.8214 - val_loss: 0.4209
Epoch 2/10
3269/3269 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8275 - loss: 0.4044 - val_accuracy: 0.8499 - val_loss: 0.3625
Epoch 3/10
3269/3269 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8526 - loss: 0.3512 - val_accuracy: 0.8641 - val_loss: 0.3274
Epoch 4/10
3269/3269 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8707 - loss: 0.3139 - val_accuracy: 0.8767 - val_loss: 0.3057
Epoch 5/10
3269/3269 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8798 - loss: 0.2910 - val_accuracy: 0.8830 - val_loss: 0.2868
Epoch 6/10
3269/3269 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.8889 - loss: 0.2719 - val_accuracy: 0.8878 - val_loss: 0.2767
Epoch 7/10
3269/3269 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.8946 - loss: 0.2578 - val_accuracy: 0.8920 - val_loss: 0.2614
Epoch 8/10
3269/3269 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9004 - loss: 0

In [10]:
model.save('my_model.h5')

In [11]:
import numpy as np

# Overfitting Experiment
batch_size = 128
single_batch_indices = np.random.choice(len(X_train), batch_size, replace=False)
X_batch = X_train.iloc[single_batch_indices]
y_batch = Y_train[single_batch_indices]

# Train on a single batch
history = model.fit(X_batch, y_batch, epochs=500, verbose=0) # Increased epochs for near-zero loss

# Validation Check
validation_loss = model.evaluate(X_test, Y_test, verbose=0)[0]
training_loss = history.history['loss'][-1]

# Get the number of parameters
num_params = model.count_params()

# Print the results
print(f"Number of parameters: {num_params}")
print(f"Final training loss: {training_loss:.4f}")
print(f"Final validation loss: {validation_loss:.4f}")


Number of parameters: 25927
Final training loss: 0.0001
Final validation loss: 0.5838
